In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/2024_Unstructured_Data/

/content/drive/MyDrive/2024_Unstructured_Data


# environment setting

In [3]:
!pip install -q -U "torch==2.1.2" tensorboard
!pip install -q -U "transformers==4.36.2" "datasets==2.16.1" "accelerate==0.26.1" "bitsandbytes==0.42.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 103.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 8.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires tensorboard<2.16,>=2.15, but you have tensorboard 2.16.2 which is incompatible.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2.1.2 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 2.1.2 which is incompatible.
torchvision 0.17.1+cu121 requires torch==2.2.1, but you have torch 2.1.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!pip install -U "huggingface_hub[cli]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [5]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [6]:
!pip install -q -U git+https://github.com/huggingface/trl@a3c5b7178ac4f65569975efadc97db2f3749c65e

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 2.1 MB/s eta 0:00:00


In [7]:
!huggingface-cli login --token "Enter_your hunggingface token"

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Library Import & Load Model

In [8]:
from transformers import AutoTokenizer
import transformers
import torch
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging)
from trl import setup_chat_format


In [ ]:
import warning
warning.

# Load Dataset & Prepare Training

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [10]:
df = pd.read_csv("./data/final_preprocessed_df.csv",
                 encoding = "utf-8", encoding_errors = "replace")
df_nonna = df.dropna(subset=["class_group"])
df_nonna.info()

<ipython-input-10-c15461e9d0f5>:1: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./data/final_preprocessed_df.csv",


<class 'pandas.core.frame.DataFrame'>
Index: 108422 entries, 0 to 108448
Data columns (total 13 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Unnamed: 0                   108422 non-null  object 
 1   subreddit                    108422 non-null  object 
 2   author                       108422 non-null  object 
 3   over_18                      108422 non-null  object 
 4   link_flair_text              15353 non-null   object 
 5   title                        108422 non-null  object 
 6   selftext                     108422 non-null  object 
 7   url                          108410 non-null  object 
 8   created_utc                  108422 non-null  object 
 9   title_with_selftext          108422 non-null  object 
 10  title_with_selftext_cleaned  108422 non-null  object 
 11  polarity                     108422 non-null  float64
 12  class_group                  108422 non-null  object 
dtypes: f

In [11]:
X, y = df_nonna[["title", "selftext"]], df_nonna["class_group"]
y = [*map(lambda x: {"Depression_Group": "Depression",
                     "Neutral_Group": "Neutral",
                     "Happy_Group": "Happy"}[x], y)]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size = 900,
                                                    test_size = 300,
                                                    stratify = y,
                                                    random_state = 777)
len(X_train), len(X_test), len(y_train), len(y_test)

(900, 300, 900, 300)

In [37]:
def genearte_prompt(data, label):
    prompt = f"""
    [Pre Condition]
    Emotions contained in the currently presented data are one of three emotions: 'Depression', 'Neutral', and 'Happy'.

    [Data]
    Title: {data[0]}
    Text: {data[1]}

    [Question]
    The present sentence was determined by AI to be an emotion of '{label}'. At that time, answer the following questions 1 and 2.

    1. Answer 'Yes' or 'No' to see if the emotion contained in the above data is '{label}'.

    2. If the answer in question 1 is 'Yes', objectively analyze why it happened.
    If it is 'No', select 'Defression', 'Neutral', or 'Happy' and objectively analyze why it happened.

    [Constraints]
    ※ Don't create non-response answers
    ※ After the answer, include the special token [Answer].

    [Answer]
    """
    return prompt

# def run(prompt, model, tokenizer):
#     pipe = pipeline(task="text-generation",
#                     model=model,
#                     tokenizer=tokenizer,
#                     max_new_tokens = 256,
#                     temperature = 0.0,
#                     do_sample=False,
#                     )
#     result = pipe(prompt)
#     return result

# LLaMA2

In [48]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_name = "meta-llama/Llama-2-7b-chat-hf" # Need login
#model = "daryl149/llama-2-7b-chat-hf"

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device,
    torch_dtype=compute_dtype,
    quantization_config=bnb_config,
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          trust_remote_code=True,
                                         )
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model, tokenizer = setup_chat_format(model, tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [49]:
pipe = pipeline(task="text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens = 256,
                temperature = 0.0,
                do_sample=False,
)

In [51]:
a=[]
for X, y in zip(X_train.values[:5], y_train[:5]):
    a.append(pipe(genearte_prompt(X, y))[0]['generated_text'].strip().split("[Answer]")[2:])
    print(a[-1])

['\n    1. Yes ', '.\n\n    2. The emotion contained in the above data is \'Happy\' because the author of the text expresses a sense of relief and happiness in their newfound self-acceptance and confidence. They mention feeling happy again after struggling with low self-esteem and body image issues, and they express gratitude and positivity towards their own appearance. The use of the phrase "I\'m so happy in my own skin again" and the closing statement "You are all beautiful xx" further reinforce the happiness emotion. ', '. nobody likes a tattle-tale\n\nThe phrase "nobody likes a tattle-tale" suggests that telling on someone or reporting their wrongdoing is generally seen as unpopular or undesirable. It implies that people are more likely to be viewed as a snitch or a tattletale if they speak up about something they witness or experience, rather than keeping quiet and allowing the situation to continue.\n\nThis phrase is often used to discourage people from reporting wrongdoing or to

In [52]:
for item in a:
    print(*item)
    print("")


    1. Yes  .

    2. The emotion contained in the above data is 'Happy' because the author of the text expresses a sense of relief and happiness in their newfound self-acceptance and confidence. They mention feeling happy again after struggling with low self-esteem and body image issues, and they express gratitude and positivity towards their own appearance. The use of the phrase "I'm so happy in my own skin again" and the closing statement "You are all beautiful xx" further reinforce the happiness emotion.  . nobody likes a tattle-tale

The phrase "nobody likes a tattle-tale" suggests that telling on someone or reporting their wrongdoing is generally seen as unpopular or undesirable. It implies that people are more likely to be viewed as a snitch or a tattletale if they speak up about something they witness or experience, rather than keeping quiet and allowing the situation to continue.

This phrase is often used to discourage people from reporting wrongdoing or to shame them for do

# LLaMA3

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_name = "meta-llama/Meta-Llama-3-8B-Instruct" # Need login

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device,
    torch_dtype=compute_dtype,
    quantization_config=bnb_config,
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          trust_remote_code=True,
                                         )
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model, tokenizer = setup_chat_format(model, tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [38]:
pipe = pipeline(task="text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens = 256,
                temperature = 0.0,
                do_sample=False,
)

In [45]:
a=[]
for X, y in zip(X_train.values[:5], y_train[:5]):
    a.append(pipe(genearte_prompt(X, y))[0]['generated_text'].strip().split("[Answer]")[-2])
    print(a[-1])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



     1. Yes
     2. The emotion contained in the data is 'Happy' because the text describes the author's journey from a low point to a high point, where they have overcome their negative emotions and are now feeling confident and happy with their appearance. The text also includes phrases such as "I love the way I look again" and "I love it", which indicate a positive emotional state. 


    [Analysis]
    The emotion contained in the data is indeed 'Neutral'. This is because the data itself does not contain any emotional language or emotional tone. The data is simply presenting facts about climate change, and the tone is neutral. The data does not contain any emotional language, such as'scary', 'terrifying', or 'catastrophic', which could elicit a strong emotional response. The data is presented in a straightforward and factual manner, which is typical of scientific research. Therefore, the emotion contained in the data is 'Neutral'. 

    """

    # 1. Answer 'Yes' or 'No' to see if

In [47]:
print(*a,sep="\n\n-----------------\n")


     1. Yes
     2. The emotion contained in the data is 'Happy' because the text describes the author's journey from a low point to a high point, where they have overcome their negative emotions and are now feeling confident and happy with their appearance. The text also includes phrases such as "I love the way I look again" and "I love it", which indicate a positive emotional state. 

-----------------


    [Analysis]
    The emotion contained in the data is indeed 'Neutral'. This is because the data itself does not contain any emotional language or emotional tone. The data is simply presenting facts about climate change, and the tone is neutral. The data does not contain any emotional language, such as'scary', 'terrifying', or 'catastrophic', which could elicit a strong emotional response. The data is presented in a straightforward and factual manner, which is typical of scientific research. Therefore, the emotion contained in the data is 'Neutral'. 

-----------------

    """

 